# Install Required Libraries


In [1]:
!pip install ultralytics
!pip install deep_sort_pytorch
!pip install opencv-python
!pip install supervision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

# Import Necessary Libraries
We import the core libraries required for detection, tracking, video handling, and drawing.

In [53]:
import cv2
import numpy as np
from ultralytics import YOLO
import os


# Load the Pretrained YOLOv8 Model
Here, we load the YOLOv8x model for more **accuracy**

In [54]:
# Load the pre-trained YOLOv8 model
model = YOLO("yolov8x.pt")

# Upload Video File to Colab
This code will open a file upload dialog for you to upload a video file (e.g., people-walking.mp4). The uploaded file will be used for detection and tracking.**You have to upload a video**

In [55]:
from google.colab import files
uploaded = files.upload()

# After upload, get the filename
video_path = list(uploaded.keys())[0]

Saving people-walking.mp4 to people-walking (4).mp4


# Initialize Video Reader and Writer
We read the video using OpenCV and extract important properties like **width, height, and FPS**. We also initialize a video writer to save the output.

In [56]:
# Open video
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Failed to open video."

# Get video properties
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Output video path and writer
out_path = "tracked_heatmap_output.avi"
video_writer = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))


# Initialize Variables for Heatmap and Counting
We prepare a heatmap canvas and define the IN and OUT lines based on frame height. We also prepare counters and memory to store object tracking paths.

In [57]:
# Heatmap canvas and counting setup
heatmap_canvas = np.zeros((h, w), dtype=np.float32)
line_up = int(h * 0.1)
line_down = int(h * 0.9)
track_memory = {}
in_count, out_count = 0, 0


# Main Video Processing Loop
This is the main loop where we:

Read each frame

Run object tracking using YOLO

Draw bounding boxes and track IDs

Update heatmap and perform IN/OUT counting

Blend the heatmap onto the frame

In [58]:
frame_id = 0
counted_in = set()
counted_out = set()
max_track_len = 30  # limit memory for each track

while True:
    ret, frame = cap.read()
    if not ret:
        break

    last_frame = frame.copy()

    # YOLOv8 tracking
    results = model.track(frame, persist=True, verbose=False)[0]

    if results.boxes is not None:
        for box in results.boxes:
            # Filter only humans (class 0 = person)
            if int(box.cls[0]) != 0:
                continue

            x1, y1, x2, y2 = map(int, box.xyxy[0])
            track_id = int(box.id.item()) if box.id is not None else 0
            cx, cy = int((x1 + x2) / 2), int((y1 + y2) / 2)

            # Draw box and ID
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            # Update heatmap - stronger heat
            cv2.circle(heatmap_canvas, (cx, cy), 20, 10, -1)

            # IN/OUT logic with duplicate count protection
            if track_id not in track_memory:
                track_memory[track_id] = []
            track_memory[track_id].append(cy)

            if len(track_memory[track_id]) > max_track_len:
                track_memory[track_id] = track_memory[track_id][-max_track_len:]

            if len(track_memory[track_id]) > 2:
                prev_y = track_memory[track_id][-2]
                if prev_y < line_up <= cy and track_id not in counted_in:
                    in_count += 1
                    counted_in.add(track_id)
                elif prev_y > line_down >= cy and track_id not in counted_out:
                    out_count += 1
                    counted_out.add(track_id)

    # Draw lines and counts
    cv2.line(frame, (0, line_up), (w, line_up), (255, 0, 0), 2)
    cv2.line(frame, (0, line_down), (w, line_down), (0, 0, 255), 2)
    cv2.putText(frame, f'IN: {in_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f'OUT: {out_count}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Heatmap overlay
    heatmap_norm = cv2.normalize(heatmap_canvas, None, 0, 255, cv2.NORM_MINMAX)
    heatmap_color = cv2.applyColorMap(heatmap_norm.astype(np.uint8), cv2.COLORMAP_JET)
    blended = cv2.addWeighted(frame, 0.7, heatmap_color, 0.3, 0) # Blending heatmap onto the frame

    video_writer.write(blended)
    frame_id += 1

# Release Resources
After processing is done, release the video and writer objects.also downloading the final heatmap image.

In [59]:
cap.release()
video_writer.release()
# Normalize and color the final heatmap
heatmap_norm = cv2.normalize(heatmap_canvas, None, 0, 255, cv2.NORM_MINMAX)
heatmap_color = cv2.applyColorMap(heatmap_norm.astype(np.uint8), cv2.COLORMAP_JET)

# Save the last frame blended with heatmap as image
last_blended = cv2.addWeighted(last_frame, 0.9, heatmap_color, 0.1, 0)
cv2.imwrite("final_heatmap_image.jpg", last_blended)
from google.colab import files
files.download("final_heatmap_image.jpg")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Download the Output Video
Finally, download the processed video from Colab to your local machine.

In [60]:
from google.colab import files
files.download(out_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>